In [18]:
import folium
import pandas as pd

# define the world map
world_map = folium.Map()

# display world map
world_map

latitude = 12.6985352
longitude = 99.7649053

# Create map and display it
san_map = folium.Map(location=[latitude, longitude], zoom_start=6)
# Display the map 
san_map

In [8]:
data2=pd.read_csv(r"D:\Onedrive\68，地理信息\15hub.csv")
# add pop-up text to each marker on the map
latitudes = list(data2.lat)
longitudes = list(data2.lng)
labels = list(data2.name)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(san_map)    
san_map



In [ ]:
san_map.save("./lines_withmarker.html")